Common imports required

In [ ]:
import requests
import time
import configparser as CP
import os
from datetime import datetime
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import drive
import json

# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.

Helper variables to the code

In [ ]:
#Add the helpers

#URL for getting the coin market list
url_tickers = 'https://api.wazirx.com/api/v2/tickers'

#Period of data extraction - We choose 60 (1 Hour)
period = 60

#Starting timestamp - (GMT: Thursday, March 8, 2018 1:00:00 PM)
startTime = 1520514000

#The wazirx API limit the number of lines per request to 2000. 
limit = 2000

#Location to store data
location = '/content/drive/MyDrive/wazirx/'
shareddrive = '/content/drive/Shareddrives/ProjectSharedDrive'

#Folder ID. Instead of giving the parent location, we can mention the folder id where you want to store the data.
# We can get the id from URL when you open the drive folder E.g. : https://drive.google.com/drive/folders/1aWbFBZuxGqNouFyxGWQkRIG5vd58JPgQ
############################&&&&&&&&&&&&&&&%^^^^^^^^^^^^^*($#))))))))))))))))))))))))))))))))))))))))))))))))))))%
fid = "1aWbFBZuxGqNouFyxGWQkRIG5vd58JPgQ"


Mount Google Drive for data extraction

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive2 = GoogleDrive(gauth)

First, we need to extract the list of markets or tickers. The key of each ticker is the coin market name.

In [ ]:
response = requests.get(url_tickers)
tickers = response.json()
markets = list(tickers.keys())

textfile = open("tickers" ,"w")
textfile.write(json.dumps(tickers))
textfile.close()

f = drive2.CreateFile({'title': "tickers", 'parents':[{'id': fid}]})
f.SetContentFile("tickers")
f.Upload()

We need to extract the details of each market and store it in the drive. 
Wazirx API limits the number of rows to 2000. We need to do multiple queries to get all the data till date.


In [ ]:
def get_data(market, period, limit, startTime):
  filename = market+".csv"

  #Create initial 2000 LINES. wazirx restrict the response to max 2000 line
  #Hence, we need to make multiple queries for getting historical data
  response = requests.get("https://x.wazirx.com/api/v2/k?market="+market+"&limit=2000"+"&period="+str(period)+"&timestamp="+str(startTime))
  out = pd.DataFrame(response.json(),columns=["Date", "Open", "High", "Low", "Close", "Volume"])
  out["Date"] =pd.to_datetime(out["Date"],unit="s")
  startTime += 2000*int(period)*60

  #create the remaining lines and append it
  while startTime < (int(time.strftime("%s"))  - 3600):
    response = requests.get("https://x.wazirx.com/api/v2/k?market="+market+"&limit=2000"+"&period="+str(period)+"&timestamp="+str(startTime))
    output = pd.DataFrame(response.json(),columns=["Date", "Open", "High", "Low", "Close", "Volume"])
    output["Date"] =pd.to_datetime(output["Date"],unit="s")

    out = out.append(output)
    startTime += 2000*int(period)*60

  out.to_csv(filename, index=False)
  f = drive2.CreateFile({'title': filename, 'parents':[{'id': fid}]})
  f.SetContentFile(filename)
  f.Upload()


In [ ]:
#Load all the files
#Ignore the files that are already downloaded in Google Drive
#It is useful when you cant finish all download in one try
uploaded = drive2.ListFile({'q': "title contains 'csv' "}).GetList()

alreadyUploaded = []
for fil in uploaded:
  alreadyUploaded.append(fil['originalFilename'][:-4])
print("There are "+ str(len(alreadyUploaded)) + " files uploaded to Google Drive")
#print(markets)
for market in markets:
  dont = 0
  if (market.find("usdt") != -1):
    for file1 in alreadyUploaded:
      if (file1 == market):
        print("Already uploaded Coin: "+market)
        dont = 1
        #print(market + " already uploaded")
        break
    if dont == 0:
      print("Downloading Coin: "+ market)
      get_data(market, period, limit, startTime)
     # print("Pending coin "+ market)
